In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
import genesis as gs
# show the iamge
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from genesis_gym import GenesisGym

[I 05/03/25 13:15:21.132 1564043] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [3]:
import random
import pathlib as pl
import numpy as np
import os, sys
import torch

## Default Args
DEFAULT_RADIUS = 0.034 # 0032
DEFAULT_HEIGHT = 0.09 # 0.1
DEFAULT_RHO = 2000
DEFAULT_FRICTION = 0.5
DEFAULT_STARTING_X = 0.65
    # parser = argparse.ArgumentParser(description='Genesis Gym Environment')
    # parser.add_argument('--vis', action='store_true', help='Enable visualization')
    # parser.add_argument('--radius', type=float, default=DEFAULT_RADIUS, help='Bottle radius')
    # parser.add_argument('-e', '--height', type=float, default=DEFAULT_HEIGHT, help='Bottle height')
    # parser.add_argument('-o', '--rho', type=float, default=DEFAULT_RHO, help='Density of the bottle')
    # parser.add_argument('--friction', type=float, default=DEFAULT_FRICTION, help='Friction of the bottle')
    # parser.add_argument('--starting_x', type=float, default=DEFAULT_STARTING_X, help='Starting x position of the bottle')
    # parser.add_argument('--max-demos', type=int, default=1e7, help='Max number of demos to load')
    # parser.add_argument('--random-agent', action='store_true', help='Use a random agent')
    # parser.add_argument('--subsample', type=int, default=2, help='Subsample ratio for the demos')
    # parser.add_argument('--env-name', type=str, default='lift', help='Environment name')
    # args = parser.parse_args()

args = {
    'vis': True,
    'radius': DEFAULT_RADIUS,
    'height': DEFAULT_HEIGHT,
    'rho': DEFAULT_RHO,
    'friction': DEFAULT_FRICTION,
    'starting_x': DEFAULT_STARTING_X,
    'max_demos': 1e7,
    'random_agent': False,
    'subsample': 2,
    'env_name': 'lift',
}

env = GenesisGym(**args)
obs = env.reset()

GenesisGym Task=lift args: {'rho': 2000, 'radius': 0.034, 'height': 0.09, 'friction': 0.5, 'vis': True, 'grayscale': False, 'time_limit': 4000, 'env_name': 'lift'}
Joint 'right_finger_tip_joint' mimics 'right_finger_bottom_joint' with multiplier -0.676 and offset -0.025
Joint 'left_finger_bottom_joint' mimics 'right_finger_bottom_joint' with multiplier -1.0 and offset 0.0
Joint 'left_finger_tip_joint' mimics 'right_finger_bottom_joint' with multiplier -0.676 and offset -0.025


[W 05/03/25 13:15:35.163 1564043] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


In [4]:
from demo_holder import GenesisDemoHolder
from collections import defaultdict
demo_player = GenesisDemoHolder(use_eef=False, subsample_ratio=args['subsample'])
TRIAL_CAN_ADJUSTED = defaultdict(lambda: False)


/home/j/workspace/fastrl/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/j/workspace/fastrl/venv/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Loaded 96 demos from /home/j/workspace/genesis_sim2real/inthewild_trials
235 (442, 7) -- 310 (917, 7) -- 317 (723, 7) -- 326 (483, 7) -- 304 (1040, 7) -- 287 (2309, 7) -- 293 (3071, 7) -- 302 (582, 7) -- 325 (730, 7) -- 311 (650, 7) -- 281 (520, 7) -- 274 (944, 7) -- 278 (334, 7) -- 289 (301, 7) -- 286 (1876, 7) -- 233 (463, 7) -- 328 (1383, 7) -- 255 (687, 7) -- 246 (1311, 7) -- 273 (734, 7) -- 236 (339, 7) -- 266 (895, 7) -- 313 (747, 7) -- 280 (362, 7) -- 327 (687, 7) -- 334 (572, 7) -- 268 (2538, 7) -- 301 (826, 7) -- 240 (772, 7) -- 250 (1675, 7) -- 298 (521, 7) -- 309 (877, 7) -- 248 (1531, 7) -- 315 (1945, 7) -- 303 (1152, 7) -- 269 (2428, 7) -- 284 (657, 7) -- 262 (468, 7) -- 238 (1275, 7) -- 277 (427, 7) -- 324 (324, 7) -- 259 (604, 7) -- 330 (1335, 7) -- 283 (660, 7) -- 275 (722, 7) -- 299 (1306, 7) -- 257 (504, 7) -- 251 (792, 7) -- 331 (1434, 7) -- 318 (759, 7) -- 307 (943, 7) -- 237 (401, 7) -- 279 (1000, 7) -- 254 (807, 7) -- 261 (706, 7) -- 314 (650, 7) -- 316 (940, 7) -

In [5]:
demo_player.next_demo()


235

In [ ]:
from genesis_gym import PZ
import time
demo_player.reset_current_demo()
env.scene.clear_debug_objects()


def get_grip_pose(env):
    # get the average position of the fingertips
    left_fingertip = env.kinova.get_link('left_finger_prox_link').get_pos().cpu().numpy()
    right_fingertip = env.kinova.get_link('right_finger_prox_link').get_pos().cpu().numpy()
    env.scene.draw_debug_sphere(left_fingertip, 0.01, color=(0, 1, 0))
    env.scene.draw_debug_sphere(right_fingertip, 0.01, color=(0, 1, 0))
    return np.mean([left_fingertip, right_fingertip], axis=0) + [0.04, 0., 0.]

trial_id = demo_player.get_trial_id()
TRIAL_CAN_ADJUSTED[trial_id] = False
env.reset(trial_id=trial_id)
action = demo_player.next_action()['action']
while action is not None:
    obs, reward, done, info = env.step(action)
    if args['vis']:
        env.render()

    # get the gripper position
    print(f"action: {action[-1]:+1.2f} {obs['state'][-1]:+1.2f}", end=' ')


    # if the gripper action is closing and the can is nearby, move the can and restart the demo
    gripper_pos = env.kinova.get_link('end_effector_link').get_pos().cpu().numpy()
    can_pose = env.bottle.get_pos().cpu().numpy()
    dp = np.linalg.norm(gripper_pos - can_pose)
    if action[-1] > 50 and dp < 0.2 and not TRIAL_CAN_ADJUSTED[trial_id] and gripper_pos[2] < 0.5:
        # get the average pos of the last 4 links 
        grip_pos = get_grip_pose(env)
        grip_pos[-1] = PZ
        # make a debug sphere
        debug_arrow = env.scene.draw_debug_arrow(pos=gripper_pos, vec=grip_pos - gripper_pos, radius=0.01, color=(1, 0, 0, 0.5))  # Green
        env.scene.draw_debug_sphere(gripper_pos, 0.01, color=(0, 1, 1))
        env.scene.draw_debug_sphere(grip_pos, 0.01, color=(0, 0, 1))

        env.reset(trial_id=trial_id)
        demo_player.reset_current_demo()
        env.set_can_to_pose(torch.Tensor(grip_pos))
        print()
        print("Gripper closing and can is nearby, restarting demo and setting can to gripper pose")
        TRIAL_CAN_ADJUSTED[trial_id] = True
        env.scene.step()

    action = demo_player.next_action()
    action = action['action'] if action is not None else None
    if done:
        break



    # break

Reset current demo 235
action: +0.52 -0.41 action: +0.52 -0.38 action: +0.52 -0.36 action: +0.52 -0.34 action: +0.52 -0.33 action: +0.52 -0.31 action: +0.52 -0.31 action: +0.51 -0.30 action: +0.49 -0.30 action: +0.49 -0.30 action: +0.49 -0.30 action: +0.49 -0.30 action: +0.49 -0.31 action: +0.49 -0.31 action: +0.50 -0.32 action: +0.50 -0.33 action: +0.50 -0.34 action: +0.51 -0.36 action: +0.51 -0.37 action: +0.51 -0.38 action: +0.51 -0.40 action: +0.51 -0.41 action: +0.51 -0.43 action: +0.51 -0.44 action: +0.51 -0.46 action: +0.51 -0.48 action: +0.51 -0.49 action: +0.51 -0.51 action: +0.51 -0.53 action: +0.51 -0.54 action: +0.51 -0.56 action: +0.51 -0.57 action: +0.51 -0.59 action: +0.51 -0.61 action: +0.51 -0.62 action: +0.51 -0.64 action: +0.51 -0.65 action: +0.51 -0.67 action: +0.51 -0.68 action: +0.51 -0.70 action: +0.51 -0.71 action: +0.51 -0.71 action: +0.51 -0.71 action: +0.51 -0.71 action: +0.51 -0.71 action: +0.51 -0.71 action: +0.51 -0.71 action: +0.51 -0.70 action: +0.51 -0.